In [1]:
import gc
import os
import sys
import json
import xgboost as xgb
from XGBoost.XGBoost_ToolBox_ver3 import *

from sklearn.datasets import *

%load_ext autoreload
%autoreload 2

c:\users\xiaowei.li\appdata\local\continuum\anaconda3\envs\myvirtualpythonenv\lib\importlib\_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
t = load_boston()

data = pd.DataFrame(load_boston().data)
data.columns = t.feature_names
target = load_boston().target

In [3]:
random_idx = np.random.choice(data.index, size=int(data.shape[0]*0.2))
print(len(random_idx))

101


In [4]:
print('Change partial data to NULL...')
data.loc[random_idx, 'AGE'] = np.nan
gc.collect()

Change partial data to NULL...


0

In [5]:
print(np.sort(data['AGE'].unique()))

[  2.9   6.    6.2   6.5   6.6   6.8   7.8   8.4   8.9   9.8   9.9  13.
  13.9  14.7  15.3  15.8  16.3  17.   17.2  17.5  17.7  17.8  18.4  18.5
  18.8  19.1  19.5  20.1  20.8  21.1  21.4  21.8  21.9  22.3  22.9  23.3
  23.4  24.8  25.8  26.3  27.6  27.7  27.9  28.1  28.4  28.8  28.9  29.2
  29.3  30.2  30.8  31.1  31.5  31.9  32.   32.2  32.3  32.9  33.   33.1
  33.2  33.3  33.5  34.1  34.2  34.5  34.9  35.7  35.9  36.1  36.6  36.8
  36.9  37.2  37.3  37.8  38.1  38.3  38.9  39.   40.   40.3  40.4  40.5
  41.1  41.5  41.9  42.2  42.3  42.4  42.6  42.8  43.7  44.4  45.   45.1
  45.4  45.6  45.7  45.8  46.7  47.2  48.   48.2  48.5  49.   49.1  49.3
  49.7  49.9  51.   51.8  51.9  52.3  52.5  52.6  52.8  52.9  53.2  53.6
  53.7  53.8  54.2  54.3  54.4  56.4  56.5  56.7  56.8  57.8  58.   58.1
  58.4  58.5  58.7  58.8  59.1  59.5  59.6  59.7  61.1  61.5  61.8  62.2
  62.5  62.8  63.   63.1  65.1  65.2  65.3  65.4  66.1  66.2  66.5  66.6
  67.   67.6  67.8  68.1  68.2  69.1  69.5  69.6  69

In [6]:
myxgb = MyXGB(objective='reg:linear')

In [7]:
myxgb.get_log()

[01/24/2019 01:40:20 PM (Local Time)] INFO : Start logging...
[01/24/2019 01:40:20 PM (Local Time)] INFO : Update instance booster params to: {'colsample_bylevel': 1, 'max_delta_step': 0, 'subsample': 1, 'colsample_node': 1, 'learning_rate': 0.1, 'seed': 0, 'min_split_loss': 0, 'sketch_eps': 0.03, 'scale_pos_weight': 1, 'nthread': -1, 'colsample_bytree': 1, 'booster': 'gbtree', 'reg_lambda': 1, 'min_child_weight': 1, 'objective': 'reg:linear', 'tree_method': 'auto', 'reg_alpha': 0, 'max_depth': 6}



In [8]:
print(myxgb.xgb_default_metrics)
print(myxgb.xgb_default_objectives)

['rmse', 'mae', 'logloss', 'error', 'error@t', 'merror', 'mlogloss', 'auc', 'ndcg', 'map', 'map@n', 'map@n-', 'poisson-nloglik', 'gamma-nloglik', 'cox-nloglix', 'gamma-deviance', 'tweedie-nloglik']
['reg:linear', 'reg:logistic', 'binary:logistic', 'binary:logitraw', 'binary:hinge', 'count:poisson', 'survival:cox', 'multi:softmax', 'multi:softprob', 'rank:pairwise', 'rank:ndcg', 'rank:map', 'reg:gamma', 'reg:tweedie']


In [9]:
num_boost_round=500

learning_rates = [0.1, 0.001]

for i in range(num_boost_round-len(learning_rates)):
    learning_rates.append(0.0001)

In [10]:
model = myxgb.fit(data, target, learning_rate=0.001, verbose_eval=200, eval_metric=['rmse', 'mae'], num_boost_round=1000, nthread=-1, early_stopping_rounds=30)

model training
[0]	Train-rmse:24.002	Train-mae:22.0741	Valid-rmse:23.4891	Valid-mae:21.8644
Multiple eval metrics have been passed: 'Valid-mae' will be used for early stopping.

Will train until Valid-mae hasn't improved in 30 rounds.
[200]	Train-rmse:19.8702	Train-mae:18.1567	Valid-rmse:19.3849	Valid-mae:17.9037
[400]	Train-rmse:16.4869	Train-mae:14.9418	Valid-rmse:16.0489	Valid-mae:14.6566
[600]	Train-rmse:13.7137	Train-mae:12.302	Valid-rmse:13.3111	Valid-mae:11.9779
[800]	Train-rmse:11.4404	Train-mae:10.1403	Valid-rmse:11.0844	Valid-mae:9.81965
[999]	Train-rmse:9.58531	Train-mae:8.38152	Valid-rmse:9.3293	Valid-mae:8.12349
*********************************************
For model training, Time cost: 1.81 second(s)
*********************************************


In [11]:
auto_model = myxgb.auto_fit(data, target, learning_rate=0.001, learning_rate_tolerance=0.000001, verbose_eval=500, eval_metric=['rmse'], \
                            num_boost_round=10000, nthread=-1, early_stopping_rounds=100)

model training
[0]	Train-rmse:24.002	Valid-rmse:23.4891
Multiple eval metrics have been passed: 'Valid-rmse' will be used for early stopping.

Will train until Valid-rmse hasn't improved in 100 rounds.
[500]	Train-rmse:15.0316	Valid-rmse:14.6116
[1000]	Train-rmse:9.57682	Valid-rmse:9.32103
[1500]	Train-rmse:6.2212	Valid-rmse:6.29602
[2000]	Train-rmse:4.13906	Valid-rmse:4.61131
[2500]	Train-rmse:2.8528	Valid-rmse:3.75606
[3000]	Train-rmse:2.07122	Valid-rmse:3.40776
[3500]	Train-rmse:1.60245	Valid-rmse:3.27456
[4000]	Train-rmse:1.30142	Valid-rmse:3.21165
[4500]	Train-rmse:1.10877	Valid-rmse:3.1812
Stopping. Best iteration:
[4521]	Train-rmse:1.10287	Valid-rmse:3.18056

*********************************************
For model training, Time cost: 5.47 second(s)
*********************************************
For learning rate of 0.001, best score is 3.180562
model training
[0]	Train-rmse:1.07143	Valid-rmse:3.18085
Multiple eval metrics have been passed: 'Valid-rmse' will be used for early sto

In [12]:
# res = myxgb.xgb_cv(data, target, learning_rate=0.001, metrics=['rmse'], 
#                    early_stopping_rounds=50, num_boost_round=10000, verbose_eval=500)

In [13]:
myxgb.bayes_opt_X = data
myxgb.bayes_opt_y = target
myxgb.bayes_opt_metrics = 'rmse'

In [14]:
# scores = myxgb.bayes_tuning()